In [3]:
import requests
import json
import base64
import PIL
from PIL import Image, ImageOps
import pandas as pd
import time
import statsmodels.api as sm

In [10]:

BASE_URL = 'https://api-us.faceplusplus.com/facepp/v3/detect'

def detect(image_path):
    """
    This function uses HTTP POST to query the Face++ API 
    that detects and classifies faces in the given image
    
    Args:
        image_path: path to the image that we want to classify
        
    Returns:
        result: a dictionary containing the API response
    """

    #read the image
    im = Image.open(image_path)
    #flip image
    out = ImageOps.grayscale(im)
    split = image_path.split("/")
    split[-1] = "grayscale-" + split[-1]
    split[-2] = "grayscale"
    transp_image_path = "/".join(split)
    out.save(transp_image_path)
    
    
    def get_call(image_path): 
        with open(image_path, 'rb') as image_file:
            # encode in base64
            image_base64 = base64.b64encode(image_file.read())
            
            return {'api_key': "AJUC6jZcYaEtVm62lA944PD18g8qzLn-",
                'api_secret': "NiML8ojHjroKMog2AKvzMuu5PpaJhDP2",
                'image_base64': image_base64,
                'return_attributes': 'gender,age'} 
    
    
    original_response = requests.post(url = BASE_URL, data = get_call(image_path))
    flipped_response = requests.post(url = BASE_URL, data = get_call(transp_image_path))
    return json.loads(original_response.content), json.loads(flipped_response.content)

In [13]:
def hit_fpp_api(directory):
    original_ages = []
    original_genders = []
    transformed_ages = []
    transformed_genders = []
    images = []
    for image in os.listdir(f"{directory}/original"):
        if image != ".DS_Store":
            images.append(image)
            original, flipped  = detect(f"{directory}/original/{image}")
            original_ages.append(original['faces'][0]['attributes']['age']['value'])
            original_genders.append(original['faces'][0]['attributes']['gender']['value'])
            transformed_ages.append(flipped['faces'][0]['attributes']['age']['value'])
            transformed_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
            time.sleep(2)
    df = pd.DataFrame()
    df["Images"] = images
    df["Original Age"] = original_ages
    df["Original Gender"] = original_genders
    df["Y-Axis Flip Age"] = transformed_ages
    df["Y-Axis Flip Gender"] = transformed_genders
    return df

In [14]:
style_gan_df = hit_fpp_api("../stock_images")

In [15]:
style_gan_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender
0,c_2_0_3.jpg,36,Female,31,Male
1,c_2_2_1.jpg,22,Female,25,Male
2,d_0_0_2.jpg,48,Female,54,Female
3,e_2_0_4.jpg,80,Female,58,Female
4,d_0_2_0.jpg,16,Male,46,Female
...,...,...,...,...,...
95,a_0_2_1.jpg,21,Male,23,Male
96,a_0_2_3.jpg,53,Male,52,Male
97,a_0_0_1.jpg,25,Female,24,Female
98,a_0_0_0.jpg,34,Female,40,Female


In [19]:
style_gan_df.to_csv("grayscale_stylegan_data.csv")

In [16]:
stock_df = hit_fpp_api("../stylegan_images")

In [17]:
stock_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender
0,1628188590507_2_2_3.jpg,66,Male,65,Male
1,1628188590507_2_0_1.jpg,21,Female,21,Female
2,1628188590507_2_0_0.jpg,20,Female,20,Female
3,1628188590507_2_2_2.jpg,31,Male,30,Male
4,1628188590507_2_2_0.jpg,19,Female,19,Female
...,...,...,...,...,...
95,1628188536814_0_2_4.jpg,80,Male,80,Male
96,1628188536814_0_0_4.jpg,83,Female,82,Female
97,1628188755886_2_0_4.jpg,75,Female,68,Female
98,1628188704081_2_0_4.jpg,82,Male,76,Female


In [ ]:
stock_df.to_csv("grayscale_stock_data.csv")

In [ ]:
sg_original_ages = []
sg_original_genders = []
sg_y_flipped_ages = []
sg_y_flipped_genders = []
for stylegan_image in os.listdir("stylegan_images"):
    if stylegan_image != ".DS_Store" and "transposed" not in stylegan_image:
        original, flipped  = detect(f"stylegan_images/{stylegan_image}")
        sg_original_ages.append(original['faces'][0]['attributes']['age']['value'])
        sg_original_genders.append(original['faces'][0]['attributes']['gender']['value'])
        sg_y_flipped_ages.append(flipped['faces'][0]['attributes']['age']['value'])
        sg_y_flipped_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
        time.sleep(2)

In [ ]:
stock_original_ages = []
stock_original_genders = []
stock_y_flipped_ages = []
stock_y_flipped_genders = []
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store" and "transposed" not in stock_image:
        original, flipped  = detect(f"stock_images/{stock_image}")
        stock_original_ages.append(original['faces'][0]['attributes']['age']['value'])
        stock_original_genders.append(original['faces'][0]['attributes']['gender']['value'])
        stock_y_flipped_ages.append(flipped['faces'][0]['attributes']['age']['value'])
        stock_y_flipped_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
        time.sleep(2)